# **Computational Theory Assessment (2024/2025)**
Ronan Francis (G00403092)
Computational Theory

## **Table of Contents**
1. [Introduction](#introduction)
2. [Task 1: Binary Representations](#task-1-binary-representations)
3. [Task 2: Hash Functions](#task-2-hash-functions)
4. [Task 3: SHA256 Padding](#task-3-sha256-padding)
5. [Task 4: Prime Numbers](#task-4-prime-numbers)
6. [Task 5: Roots](#task-5-roots)
7. [Task 6: Proof of Work](#task-6-proof-of-work)
8. [Task 7: Turing Machines](#task-7-turing-machines)
9. [Task 8: Computational Complexity](#task-8-computational-complexity)
10. [Conclusion](#conclusion)

## **Introduction**
...

## **Task 1: Binary Representations**
...

[Back to Top](#table-of-contents)

## **Task 2: Hash Functions**
...

[Back to Top](#table-of-contents)

## **Task 3: SHA256**
...

[Back to Top](#table-of-contents)

## **Task 4: Prime Numbers**
...

[Back to Top](#table-of-contents)

## **Task 5: Roots**
...

[Back to Top](#table-of-contents)

## **Task 6: Proof of Work**
...

[Back to Top](#table-of-contents)

## **Task 7: Turing Machines**
...

[Back to Top](#table-of-contents)

## **Task 8: Computational Complexity**
...

[Back to Top](#table-of-contents)

## **Conclusion**
...

[Back to Top](#table-of-contents)